In [271]:
import numpy as np 
import pandas as pd 
import nltk
from nltk.corpus import stopwords
import string

from string import punctuation
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer


In [272]:
cd D:\Shakeer\Kaggle projects\nlp-getting-started

D:\Shakeer\Kaggle projects\nlp-getting-started


In [273]:
data=pd.read_csv("train.csv")
data_test=pd.read_csv("test.csv")

In [274]:
data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [275]:
data_test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [276]:
data.shape

(7613, 5)

In [277]:
data_test.shape

(3263, 4)

In [278]:
data.drop_duplicates(inplace = True)
data_test.drop_duplicates(inplace = True)

In [279]:
data.shape

(7613, 5)

In [284]:
data_test.shape

(3263, 4)

In [285]:
tweet_tok=TweetTokenizer()

In [286]:
lemm=WordNetLemmatizer()

In [291]:
def clean_txt(sent):
    stemmer_s = SnowballStemmer("english")
    tokens = word_tokenize(sent.lower())
    stop_updated = stopwords.words("english") + list(punctuation) + ["..."]
    text = [stemmer_s.stem(term) for term in tokens if term not in stop_updated and len(term) > 2]
    #text = [lemm.lemmatize(term) for term in tokens if term not in stop_updated and len(term) > 2]
    #text = [term for term in tokens if term not in stop_updated and len(term) > 2] 
    res = " ".join(text)
    return res

In [292]:
data['cleared_text'] = data.text.apply(clean_txt)

In [293]:
data.head()

,id,keyword,location,text,target,cleared_text
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deed reason earthquak may allah forgiv
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near rong sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,resid ask shelter place notifi offic evacu she...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,"13,000 peopl receiv wildfir evacu order califo..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,got sent photo rubi alaska smoke wildfir pour ...


In [294]:
def clean_text(sent):
    stemmer_s = SnowballStemmer("english")
    tokens = word_tokenize(sent.lower())
    stop_updated = stopwords.words("english") + list(punctuation) + ["..."]
    text = [stemmer_s.stem(term) for term in tokens if term not in stop_updated and len(term) > 2]
    #text = [lemm.lemmatize(term) for term in tokens if term not in stop_updated and len(term) > 2]
    #text = [term for term in tokens if term not in stop_updated and len(term) > 2] 
    res = " ".join(text)
    return res

In [295]:
data_test['cleared_text'] = data_test.text.apply(clean_text)

In [296]:
data_test.head()

,id,keyword,location,text,cleared_text
0,0,NaN,NaN,Just happened a terrible car crash,happen terribl car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",heard earthquak differ citi stay safe everyon
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",forest fire spot pond gees flee across street ...
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,apocalyps light spokan wildfir
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,typhoon soudelor kill china taiwan


In [297]:
x=data.cleared_text.values
y=data.target.values

In [298]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=3500)
x_train_tfidf = tfidf_vectorizer.fit_transform(x)
x_test_tfidf = tfidf_vectorizer.transform(data_test.cleared_text.values)

In [299]:
print(x_train_tfidf.shape,x_test_tfidf.shape)

(7613, 3500) (3263, 3500)


In [300]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(x_train_tfidf, y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [301]:
y_pred = classifier.predict(x_test_tfidf)

In [302]:
output=pd.DataFrame(y_pred,data_test['id'])

In [303]:
output.rename(columns={0:'target'},inplace=True)

In [304]:
output.reset_index(inplace=True)

In [305]:
output.columns

Index(['id', 'target'], dtype='object')

In [306]:
output.head()

,id,target
0,0,1
1,2,0
2,3,1
3,9,1
4,11,1


In [307]:
output.to_csv('NLP_final_submission3.csv')